# Análisis de sentimientos

Index:
1. **Detección de elementos de opiniones**: Identificación de targets, aspectos y opinion words.
2. **Detección de polaridad**: Extracción de polaridades de las opiniones.
3. **Clasificación**: clasificación de opiniones.
4. **Evaluación**: comparación de modelos.


### Adonis González Godoy

## 1. Detección de elementos de opiniones 

Detección de targets y aspectos en opiniones.

Primero, cargamos los paquetes necesarios y los datos.

In [1]:
import pandas as pd
import gensim
import nltk
import re

hotel_opinions_data = pd.read_csv('hotel_opinions.csv')

hotel_opinions_data.head()

,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,id10326,The room was kind of clean but had a VERY stro...,Edge,Mobile,not happy
1,id10327,I stayed at the Crown Plaza April -- - April -...,Internet Explorer,Mobile,not happy
2,id10328,I booked this hotel through Hotwire at the low...,Mozilla,Tablet,not happy
3,id10329,Stayed here with husband and sons on the way t...,InternetExplorer,Desktop,happy
4,id10330,My girlfriends and I stayed here to celebrate ...,Edge,Tablet,not happy


Cargamos el modelo Word2Vec del Google News dataset, con aproximadamente cien mil millones de palabras. <b>el modelo ocupa casi 2 GB! y puede estar unos minutos para cargarse </b>

In [14]:
import gensim.downloader as api
wv = api.load('word2vec-google-news-300')

Convertimos los comentarios en un solo texto en minúscula

In [2]:
comment_text = " ".join(hotel_opinions_data['Description'].to_list()).lower()

print(comment_text[:100])

the room was kind of clean but had a very strong smell of dogs. generally below average but ok for a


Obtendremos los sintagmas nominales de los comentarios. Los sintagmas nominales serán los candidatos a ser los targets y los aspectos de las opiniones


<li>Obtenemos las phrases del texto de comentarios con el modelo de phrases</li>


In [3]:
import os, sys
from gensim.models.phrases import Phrases
from nltk import word_tokenize

In [4]:
ROOT_DIR = os.path.abspath("../../")
sys.path.append(ROOT_DIR)
model_path = os.path.join(ROOT_DIR, "PRAC1\Practica1\model_phrases")

# cargamos el modelo
model_phrases = Phrases.load(model_path)

In [5]:
doc_tokens = word_tokenize(comment_text)
text_phrases = model_phrases[doc_tokens]

print(text_phrases[:100])

['the', 'room', 'was', 'kind_of', 'clean', 'but', 'had_a', 'very', 'strong', 'smell', 'of', 'dogs', '.', 'generally', 'below', 'average', 'but', 'ok', 'for_a', 'overnight', 'stay', 'if_you', "'re", 'not', 'too', 'fussy', '.', 'would', 'consider', 'staying', 'again', 'if', 'the_price', 'was', 'right', '.', 'breakfast', 'was', 'free', 'and', 'just', 'about', 'better_than', 'nothing', '.', 'i', 'stayed', 'at_the', 'crown', 'plaza', 'april', '--', '-', 'april', '--', ',', '--', '--', '.', 'the', 'staff', 'was', 'friendly', 'and', 'attentive', '.', 'the', 'elevators', 'are', 'tiny', '(', 'about', '-', "'", 'by', '-', "'", ')', '.', 'the', 'food', 'in_the', 'restaurant', 'was', 'delicious', 'but', 'priced', 'a_little', 'on_the', 'high', 'side', '.', 'of', 'course', 'this_is', 'washington', 'dc', '.', 'there_is', 'no']


<ul>
<li>Las phrases pueden contener sintagmas nominales (e.g: at_night). La función <i>get_np</i> sirve para sacar el sintagma nominal de la phrase (e.g: at_night -> night).</li>
</ul>

In [6]:
def get_np(candidate):
    np = ''
    tokens = candidate.split('_')
    try:
        tagged_tokens = nltk.pos_tag(tokens)
        PoS_initial = tagged_tokens[0][1][:2]
        PoS_final = tagged_tokens[-1][1][:2]
        if PoS_initial == 'NN':
            if len(tagged_tokens) > 1:
                if PoS_final == 'NN':
                    np = candidate
                else:
                    np = tokens[0]
            else:
                np = candidate
        else:
            if PoS_final == 'NN':
                np = tokens[-1]
    except:
        print(candidate,"CANNOT BE PoS TAGGED")
    return np

<ul>
<li>Extraemos los sintagmas nominales de las phrases con la función get_np. Así obtendremos todos los candidatos a target y a aspecto en las opiniones.
</ul>

In [7]:
import pickle

In [8]:
res = [get_np(x) for x in text_phrases if get_np(x) != ""]

In [ ]:
# serializamos los resultados
with open("NPs_COMMENTS.txt", 'wb') as fp:
    pickle.dump(res, fp)

In [9]:
# just to deserialize
with open ('NPs_COMMENTS.txt', 'rb') as fp:
    res = pickle.load(fp)

In [10]:
print(len(res))

2048820


Cargamos los sintagmas nominales de los comentarios que has guardado en un archivo. En un fichero llamado 'NPs_COMMENTS.txt'

<li>Obtenemos targets que están en las opiniones. Para obtenerlos, encontraremos los 100 sintagmas nominales más parecidos al término <i>hotel</i> según el modelo Word2Vec de Google News que también están las opiniones</li>

In [11]:
term = "hotel"

In [51]:
sims = wv.most_similar(term, topn=5000)
result_terms = [sims[w][0] for w in range(0, len(sims)) if sims[w][0] in res]

In [54]:
targets = result_terms[:100]
pd.DataFrame(list(targets), columns=['Target term'])

,Target term
0,hotels
1,motel
2,guesthouse
3,resort
4,hotelier
5,restaurant
6,inn
7,lodgings
8,villa
9,hoteliers


<ul>
<li>También obtendremos los targets calculando los sintagmas nominales de las opiniones que son más cercanos al synset hotel.n.01, según la distancia de Wu y Palmer. Para calcular la proximidad de un sintagma nominal, coge el synset de su primera acepción (e.g: 'dog.n.01'). Ver los 50 synsets más próximos</li>
</ul>

In [127]:
from nltk.corpus import wordnet as wn

In [187]:
targ_wp = []
hotel = wn.synset('hotel.n.01')

for i in res[:90000]:
    try:
        tmp = wn.synset(str(i)+'.n.01')
        if hotel.wup_similarity(tmp) > 0.75:
            if (i, hotel.wup_similarity(tmp)) not in targ_wp:
                targ_wp.append((i, hotel.wup_similarity(tmp)))
    except:
        pass

In [205]:
targe_wu_pal = pd.DataFrame(list(targ_wp), columns=['Target', "Dist Wu Pa"])
targe_wu_pal.sort_values('Dist Wu Pa', ascending=False).reset_index(drop=True)

,Target,Dist Wu Pa
0,hotel,1.000000
1,fleabag,0.941176
2,inn,0.941176
3,hostel,0.941176
4,resort,0.941176
5,building,0.933333
6,motel,0.888889
7,roadhouse,0.888889
8,restaurant,0.875000
9,eatery,0.875000


<hr></hr>

Algunos de los targets que no se ha podido apreciar en el modelo Word2Vec son: `fleabag`, `roadhouse`, `cottage`, `housing`, entre otros.

<hr></hr>

<li>Encontraremos los aspectos vinculados a los targets según el modelo Word2Vec. Lo haremos encontrando los sintagmas nominales de los comentarios más cercanos semánticamente a los targets. Los targets deben estar en su forma singular y plural. Para calcular la distancia semántica lo haremos con la función <i>model_w2v.similarity</i>. Estableceremos un umbral de similitud a partir del cual obtendremos más aspectos introduciendo el menos ruido posible</li>

In [61]:
# iteramps cada sintagma nominal de los comentarios,
# comparando la distancia semantica con los targets
# guardamos los sintagmas que superen el threshold
# result -> tupla (sn, targ, dist)

w2v_tuples = []
threshold = 0.6
for i in res[:10000]: # 10000 cause time exceeded 
    for j in targets:
        try:
            if i != j and wv.similarity(j, i) > threshold: 
                w2v_tuples.append((i, j, wv.similarity(j, i)))
        except:
            pass

In [62]:
result_close = [w2v_tuples[w] for w in range(0, len(w2v_tuples))]
pd.DataFrame(list(result_close), columns=['SN', 'Targ', 'Dist'])

,SN,Targ,Dist
0,room,rooms,0.760579
1,restaurant,restuarant,0.792897
2,restaurant,resturant,0.694312
3,restaurant,steakhouse,0.726985
4,restaurant,restaurants,0.772289
...,...,...,...
2154,motel,apartment,0.604717
2155,motel,motels,0.668736
2156,store,mall,0.640925
2157,room,rooms,0.760579


<hr></hr>

Los sintagmas que tienen diferencias gramaticales, por ejemplo entre el sintagma `restaurant` y el target `restuarant`, además algunos otros `store`y `mall` solo se ha deteterminado la similutud de la palabra dado un umbral.

Tal vez usando **ConcepNet** se podría obtener mejores resultado ya que esta diseñada para realizar inferencias prácticas basadas en el contexto sobre texto del mundo real.

<hr></hr>

<li>Comrpobamos si es verdad o no que hay targets y aspectos que varían según el tipo de dispositivo en el que el opinador escribe su comentario. Por ejemplo, si utiliza un 'Desktop' o un 'Mobile'</li>


In [104]:
opin_desktop = [x for x in hotel_opinions_data[hotel_opinions_data['Device_Used'] == 'Desktop']['Description']]
opin_mobile = [x for x in hotel_opinions_data[hotel_opinions_data['Device_Used'] == 'Mobile']['Description']]

In [105]:
comment_desktop = " ".join(opin_desktop).lower()
comment_mobile = " ".join(opin_mobile).lower()

In [112]:
doc_tokens_desktop = word_tokenize(comment_desktop)
doc_tokens_mobile = word_tokenize(comment_mobile)

In [125]:
sim_both = wv.most_similar(term, topn=1000)

desk = [sim_both[w][0] for w in range(0, len(sim_both)) if sim_both[w][0] in doc_tokens_desktop]
mobil = [sim_both[w][0] for w in range(0, len(sim_both)) if sim_both[w][0] in doc_tokens_mobile]

In [126]:
pd.DataFrame([(desk, mobil) for desk, mobil in zip(desk,mobil)], columns=['Desktop', 'Mobile'])

,Desktop,Mobile
0,hotels,hotels
1,motel,motel
2,guesthouse,resort
3,resort,lodging
4,lodging,hotelier
5,hotelier,restaurant
6,restaurant,inn
7,inn,lodgings
8,lodgings,villa
9,villa,hoteliers


Se buscan unos cuantos términos y se encuentra algunas diferencias importantes como: los usuarios de escritorio suelen usar `guesthouse` en lugar `resort` como lo hacen los de Móbil, otra diferencia importante es `guestroom ` vs `fivestars` de lo usuarios de móbil.

Por lo tanto sí se puede demostrar que hay diferencias importantes entre los usuarios desktop y mobile.

# 2. Detección de polaridad

### 2.1. Obtención y preprocesado de datos

En este apartado obtendremos los datos textuales necesarios para estudiar la opinión sobre un tema concreto.

<li>Obtendremos 1.000 twits sobre el tema #covid con la librería tweepy.</li>

In [1]:
import tweepy as tw
from tweepy import API
from tweepy import OAuthHandler

In [3]:
#link  https://dev.twitter.com/apps/

CONSUMER_KEY = ""
CONSUMER_SECRET = ""

ACCESS_KEY = "-"
ACCESS_SECRET = ""

auth = tw.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_KEY, ACCESS_SECRET)

api = tw.API(auth)

In [333]:
search = "#covid"
date_since = "2015-01-01"
num_tweets = 1000

In [334]:
tws = []

tweets = tw.Cursor(api.search, q=search, lang ='en', 
                   tweet_mode="extended").items(num_tweets)
    
for tweet in tweets:
    if "retweeted_status" in dir(tweet):
        tws.append(tweet.retweeted_status.full_text)
    else:
        tws.append(tweet.full_text)

In [336]:
# just to serialize
with open("tweets.txt", 'wb') as fp:
    pickle.dump(tws, fp)

In [63]:
# just to deserialize
with open ('tweets.txt', 'rb') as fp:
    tws = pickle.load(fp)

In [78]:
print(f'Twits recuperados: {len(tws)}')

Twits recuperados: 1000


<li>Preprocesamos los twits eliminando caracteres extraños, emojis, urls y todo lo que creas conveniente para unificar el texto. A continuación se indican los patterns de emoticonos, y otros símbolos que hay que compilar para hacer la limpieza.</li>

In [66]:
import re

emoji_pattern = re.compile("["
        u"\U0001F600-\U0001F64F"  # emoticons
        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
        u"\U0001F680-\U0001F6FF"  # transport & map symbols
        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           "]+", flags=re.UNICODE)

In [67]:
tws_clean = []
for i in tws:
    tws_clean.append(re.sub(r'(\s)http\S+|[#@&;~\n]', '', emoji_pattern.sub(r' ', i)))

In [70]:
comment_tws = " ".join(tws_clean).lower()

### 2.2. Análisis de Sentimientos

<li>Utilizamos el diccionario de opinión words (archivo AFINN-111) para extraer la polaridad de un twit como la media de las opinión words del twit que estén en el diccionario.</li>

In [79]:
word_dict = {}
with open('AFINN-111.txt', encoding='UTF-8') as fid:
    for n, line in enumerate(fid):
        word, score = line.strip().split('\t')
        word_dict[word] = int(score)

In [80]:
tws_tokens = [w for w in word_tokenize(comment_tws)]

In [83]:
result = 0
for i in word_dict:
    for j in tws_tokens:
        if (i == j):
            result =+ word_dict[i]

In [82]:
print(f'Resultado de polaridad {result}')

Resultado de polaridad 1


<li>El archivo Emoji-Polarity contiene un documento similar a AFINN-111 pero para emojis. Úsalo para extraer la polaridad de los twits analizados que contengan emojis. Calculamos la polaridad como y se demuestra si cambia la polaridad según si consideramos los emojis o no.</li>
</ul>

In [84]:
import json
import codecs

data = json.load(codecs.open('emoji-polarity.txt', 'r', 'utf-8-sig'))
data["emojis"][1]['emoji']

'😠'

In [85]:
comment_tws_emoj = " ".join(tws).lower()
tokens_emoji_inc = [w for w in word_tokenize(comment_tws_emoj)]

In [86]:
res_pol = 0
for x in tokens_emoji_inc:
    for i, j in enumerate (data["emojis"]):
        if data["emojis"][i]['emoji'] == x:
            res_pol =+ data["emojis"][i]['polarity']

In [88]:
print(f'La polaridad es: {res_pol}')

La polaridad es: -4


<hr></hr>
Se puede ver que la polaridad ha cambiado negativamente usando los emojis, la mayoría de personas que han usado emojis lo han hecho con un sentimiento negativo, mientras que sin usar los emojis solo se podía apreciar un sentimiento neutro.
<hr></hr>

## 3. Clasificación


Volveremos a utilizar los comentarios de hoteles para crear un clasificador de opiniones en happy y not_happy.

In [89]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [90]:
hotel_opinions_data = pd.read_csv('hotel_opinions.csv')

<ul>
<li>Vectorizamos las opiniones con el tf.idf vectorizer y entrena el clasificador usando la logistic regression</li>
</ul>

In [91]:
hap_unh = hotel_opinions_data[
    (hotel_opinions_data['Is_Response'] == 'happy') | 
    (hotel_opinions_data['Is_Response'] == 'not happy')]

hap_unh_text = hap_unh['Description']

In [92]:
len(hap_unh_text)

38932

In [93]:
# default word analyzer
vectorizer = TfidfVectorizer()

In [94]:
#Aplicamos el vectorizador a los textos
X = vectorizer.fit_transform(hap_unh_text[:10000])

# Matriz con los vectores
M = X.toarray()

In [95]:
data_labels = []

data = hap_unh['Is_Response']

for i in data[:10000]:
    if i == 'happy':
        data_labels.append(0)
    else:
        data_labels.append(1)

X_train, X_test, y_train, y_test  = train_test_split(
        M, 
        data_labels,    
        train_size=0.80, 
        random_state=1234)

In [96]:
#Modelo elegido: logistic regression
logreg_model = LogisticRegression()

#Entrenamiento
logreg_model.fit(X=X_train, y=y_train)

#Clasificación
y_pred = logreg_model.predict(X_test)

#Métricas de evaluación
bm = metrics.classification_report(y_test, y_pred, labels=[0,1])
print(bm)

              precision    recall  f1-score   support

           0       0.87      0.94      0.90      1378
           1       0.85      0.68      0.76       622

    accuracy                           0.86      2000
   macro avg       0.86      0.81      0.83      2000
weighted avg       0.86      0.86      0.86      2000



## 4. Evaluación: Comparación de modelos

<li>Vectorizamos las opiniones con el tf.idf vectorizer, pero transformando las opiniones de manera que las palabras estén lematizadas y no haya stopwords.</li>
<li>Entrenamos el clasificador con estos vectores usando la logístic regression</li>

In [97]:
def text_process(text):
    tokens = word_tokenize(text.lower())
    tokens_stripped = [ts.strip('".,;:-():!?-‘’ ') for ts in tokens]
    nopunc = ' '.join(tokens_stripped)
    return [word for word in nopunc.split() if word not in stopwords]

In [98]:
stopwords = nltk.corpus.stopwords.words('english')

In [99]:
vectorizer = TfidfVectorizer(analyzer= text_process)


In [100]:
#Aplicamos el vectorizador a los textos
X = vectorizer.fit_transform(hap_unh_text[:10000])
#Matriz con los vectores
M = X.toarray()

In [101]:
data_labels = []

data = hap_unh['Is_Response']

for i in data[:10000]:
    if i == 'happy':
        data_labels.append(0)
    else:
        data_labels.append(1)

X_train, X_test, y_train, y_test  = train_test_split(
        M, 
        data_labels,    
        train_size=0.80, 
        random_state=1234)

In [102]:
#Modelo elegido: logistic regression
logreg_model = LogisticRegression()

#Entrenamiento
logreg_model.fit(X=X_train, y=y_train)

#Clasificación
y_pred = logreg_model.predict(X_test)

#Métricas de evaluación
bm = metrics.classification_report(y_test, y_pred, labels=[0,1])
print(bm)

              precision    recall  f1-score   support

           0       0.85      0.96      0.90      1378
           1       0.86      0.64      0.73       622

    accuracy                           0.86      2000
   macro avg       0.86      0.80      0.82      2000
weighted avg       0.86      0.86      0.85      2000



Las máquinas de vectores de soporte podrían buscar en un hiperplano que separe de forma óptima los puntos de la clase `happy` de los `not happy`.  